In [251]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import *
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import StandardScaler
from pyspark.sql.window import Window
from pyspark.sql.functions import percent_rank
from pyspark.ml.functions import vector_to_array
from pyspark.ml.feature import VectorAssembler
import numpy as np


spark = (
    SparkSession.builder.appName("ADS project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

In [252]:
df_old = spark.read.parquet('../data/curated/merchant_consumer_abs')

In [253]:
df_old

postcode,total_earners,median_age,income_sum,2021_population,km2,income_mean,persons/km2,merchant_name,revenue_level,user_id,order_datetime,products,take_rate,tag,category,dollar_value,order_year,order_month,order_day,consumer,consumer_address,consumer_state,consumer_postcode,consumer_gender,fraud_group,__index_level_0__
0800,5631.999973601934,36.75,4.20609029345557E8,7678.999968193022,3.199999994700948,74682.00129918633,2399.687494034091,Elit Sed Consequa...,a,10413,2022-04-27,artist supply and...,5.89,artist supply craft,art and gifts,375.16773164703153,2022,4,27,Kelly Clayton,4211 Rodney Tunne...,NT,0800,Female,0,0
0800,5631.999973601934,36.75,4.20609029345557E8,7678.999968193022,3.199999994700948,74682.00129918633,2399.687494034091,Elit Sed Consequa...,a,22246,2022-10-07,artist supply and...,5.89,artist supply craft,art and gifts,617.8791313462011,2022,10,7,Corey Estrada,1703 Boyd Shore,NT,0800,Male,0,1
0800,5631.999973601934,36.75,4.20609029345557E8,7678.999968193022,3.199999994700948,74682.00129918633,2399.687494034091,Elit Sed Consequa...,a,2681,2022-02-28,artist supply and...,5.89,artist supply craft,art and gifts,766.5077067424303,2022,2,28,Nicole Bishop,467 Robert Island...,NT,0800,Female,0,2
0800,5631.999973601934,36.75,4.20609029345557E8,7678.999968193022,3.199999994700948,74682.00129918633,2399.687494034091,Malesuada Vel Ltd,b,13454,2021-07-24,books periodicals...,3.56,books periodicals...,books and music,258.86595899218753,2021,7,24,Natalie Herrera,88798 Saunders Hi...,NT,0800,Female,0,3
0800,5631.999973601934,36.75,4.20609029345557E8,7678.999968193022,3.199999994700948,74682.00129918633,2399.687494034091,Varius Orci Insti...,a,10146,2021-08-08,tent and awning s...,6.3,tent awning,outdoors,4.159037931172075,2021,8,8,Shannon Mann,00817 Owens Circle,NT,0800,Female,0,4
0800,5631.999973601934,36.75,4.20609029345557E8,7678.999968193022,3.199999994700948,74682.00129918633,2399.687494034091,Volutpat Nulla In...,a,23093,2022-10-11,furniture home fu...,6.59,furniture home fu...,home and technology,8.26105405068297,2022,10,11,Robert Walker,646 Allison Ranch...,NT,0800,Male,0,5
0800,5631.999973601934,36.75,4.20609029345557E8,7678.999968193022,3.199999994700948,74682.00129918633,2399.687494034091,Mollis Integer Co...,b,13454,2022-05-02,digital goods boo...,3.96,digital goods boo...,books and music,83.63154755239155,2022,5,2,Natalie Herrera,88798 Saunders Hi...,NT,0800,Female,0,6
0800,5631.999973601934,36.75,4.20609029345557E8,7678.999968193022,3.199999994700948,74682.00129918633,2399.687494034091,Elit Sed Consequa...,a,13454,2021-04-08,artist supply and...,5.89,artist supply craft,art and gifts,87.00495112586256,2021,4,8,Natalie Herrera,88798 Saunders Hi...,NT,0800,Female,0,7
0800,5631.999973601934,36.75,4.20609029345557E8,7678.999968193022,3.199999994700948,74682.00129918633,2399.687494034091,Eros Limited,c,8092,2021-12-18,digital goods boo...,2.52,digital goods boo...,books and music,8.26187754879106,2021,12,18,Rebecca Owens,92579 Keith Neck ...,NT,0800,Female,0,8
0800,5631.999973601934,36.75,4.20609029345557E8,7678.999968193022,3.199999994700948,74682.00129918633,2399.687494034091,Hendrerit A Corpo...,a,11203,2022-07-14,watch clock and j...,6.64,watch clock jewel...,fashion and acces...,100.80643267043831,2022,7,14,John Rodriguez,563 Wallace Rue,NT,0800,Male,0,9


In [254]:
df_old.select(F.col("fraud_group") == 2).count()

11920453

## Aim: Score all the merchants based on the following four criterias.

## 1. Fraud score (0~1)

In [255]:
# The highest possible mean value of fraud is 2
# Therefore we divide the mean value by two to keep it within [-1, 1]
df_fraud = df_old.groupBy(["merchant_name"]).agg((1-(mean("fraud_group")/2)).alias("fraud_score"))

In [256]:
df_fraud.printSchema()

root
 |-- merchant_name: string (nullable = true)
 |-- fraud_score: double (nullable = true)



In [257]:
df_fraud.orderBy(F.col("fraud_score"))

merchant_name,fraud_score
Nulla Ltd,0.5
Mauris Ipsum Port...,0.5
At Lacus LLP,0.75
Eu Accumsan Sed I...,0.875
Augue Limited,0.875
Sem Magna Nec Ind...,0.9
Velit Cras Lorem ...,0.9117647058823529
Ut Ipsum Incorpor...,0.9210526315789473
Faucibus Leo Corp.,0.9444444444444444
In Mi PC,0.9444444444444444


## 2. Revenue score (0~1)

In [260]:
df_predict = spark.read.parquet("../data/curated/predictions.parquet")
df_info = spark.read.parquet('../data/meta/merchant_metadata.parquet')

In [261]:
df_category = (df_old.groupBy("merchant_name", "category").agg(max("take_rate").alias("take_rate"))).withColumnRenamed("merchant_name", "key")
df_revenue  = df_predict.groupBy(["merchant_name"]).agg(sum("prediction").alias("total_revenue"))
df_revenue = df_revenue.join(df_category, df_category.key ==  df_revenue.merchant_name,"inner").drop("key")

In [262]:
df_revenue

merchant_name,total_revenue,category,take_rate
Aliquam Auctor Ve...,169502.71718997118,outdoors,3.09
Accumsan Convalli...,691522.2894821255,books and music,6.09
Lobortis Risus LLP,141059.71258568222,outdoors,4.53
Risus Odio Auctor...,451756.56096442597,fashion and acces...,5.99
Pellentesque Indu...,52179.446049453785,outdoors,2.66
Sodales Mauris Co...,119979.0975753708,fashion and acces...,1.5
Maecenas Iaculis ...,52503.83422199742,home and technology,1.5
Sit LLP,67927.47396706758,art and gifts,5.65
Curabitur Massa LLC,52179.446049453785,outdoors,3.61
Congue A Aliquet LLC,16585.165961625524,books and music,4.07


In [263]:
df_revenue = df_revenue.withColumn("revenue_taken", F.col("total_revenue") * (F.col("take_rate") / 100)).drop("total_revenue", "take_rate")
df_revenue

merchant_name,category,revenue_taken
Aliquam Auctor Ve...,outdoors,5237.633961170109
Accumsan Convalli...,books and music,42113.70742946144
Lobortis Risus LLP,outdoors,6390.004980131404
Risus Odio Auctor...,fashion and acces...,27060.218001769117
Pellentesque Indu...,outdoors,1387.9732649154707
Sodales Mauris Co...,fashion and acces...,1799.686463630562
Maecenas Iaculis ...,home and technology,787.5575133299614
Sit LLP,art and gifts,3837.902279139318
Curabitur Massa LLC,outdoors,1883.6780023852816
Congue A Aliquet LLC,books and music,675.0162546381588


In [264]:
revenue_score = df_revenue.withColumn("a", lit("a"))

# Use the rank as the score
Window_merchant = Window.partitionBy("a").orderBy("revenue_taken")
Window_category = Window.partitionBy("category").orderBy("revenue_taken")
revenue_score = revenue_score.withColumn("merchant_rev_score", percent_rank().over(Window_merchant)) 
revenue_score = revenue_score.withColumn("category_rev_score", percent_rank().over(Window_category)) 
revenue_score = revenue_score.drop("a", "revenue_taken")

In [265]:
revenue_score

merchant_name,category,merchant_rev_score,category_rev_score
Tincidunt Aliquam...,art and gifts,0.0,0.0
Amet Ultricies Se...,art and gifts,0.0,0.0
Eleifend Nunc LLP,art and gifts,5.242463958060288E-4,0.003780718336483932
Vel Nisl Incorpor...,art and gifts,7.863695937090433E-4,0.005671077504725898
Penatibus Et Magn...,art and gifts,0.001048492791612...,0.007561436672967864
Eget Volutpat Orn...,art and gifts,0.001310615989515072,0.00945179584120983
Nulla Semper LLP,art and gifts,0.001572739187418...,0.011342155009451797
In Lorem LLP,art and gifts,0.001834862385321101,0.013232514177693762
Ut Erat PC,art and gifts,0.002096985583224...,0.015122873345935728
Volutpat Ornare LLP,art and gifts,0.00235910878112713,0.017013232514177693


Union the the fraud and revenue

In [266]:
df_fraud = df_fraud.withColumnRenamed("merchant_name", "key")
final_score = revenue_score.join(df_fraud, df_fraud.key ==  revenue_score.merchant_name,"inner").drop("key")
final_score

merchant_name,category,merchant_rev_score,category_rev_score,fraud_score
Tincidunt Aliquam...,art and gifts,0.0,0.0,1.0
Amet Ultricies Se...,art and gifts,0.0,0.0,0.9966666666666667
Eleifend Nunc LLP,art and gifts,5.242463958060288E-4,0.003780718336483932,1.0
Vel Nisl Incorpor...,art and gifts,7.863695937090433E-4,0.005671077504725898,0.9970588235294118
Penatibus Et Magn...,art and gifts,0.001048492791612...,0.007561436672967864,1.0
Eget Volutpat Orn...,art and gifts,0.001310615989515072,0.00945179584120983,1.0
Nulla Semper LLP,art and gifts,0.001572739187418...,0.011342155009451797,1.0
In Lorem LLP,art and gifts,0.001834862385321101,0.013232514177693762,0.9962406015037594
Ut Erat PC,art and gifts,0.002096985583224...,0.015122873345935728,0.9978260869565218
Volutpat Ornare LLP,art and gifts,0.00235910878112713,0.017013232514177693,1.0


## 3. Potential score (-1~1)

In [267]:
trend = spark.read.option("header", "true").csv("../data/curated/increasing.csv")
trend = trend.withColumn("monthly_increase_rate", F.col("monthly_increase_rate").cast("double"))

In [268]:
trend_positive = trend.filter(F.col("monthly_increase_rate") >= 0)
trend_negative = trend.filter(F.col("monthly_increase_rate") < 0)

In [269]:

# VectorAssembler Transformation - Converting column to vector type
assembler_trend = VectorAssembler(inputCols=["monthly_increase_rate"],outputCol="increasing_Vect")
df_trend = assembler_trend.transform(trend)
scaler = MinMaxScaler(inputCol="increasing_Vect", outputCol="trend_score")

# Standardise the trend score (since it contain negative value) as score
df_trend = scaler.fit(df_trend).transform(df_trend).drop("monthly_increase_rate","increasing_Vect").withColumn("trend_score", (2*(vector_to_array("trend_score")[0])-1))




In [270]:
df_trend = df_trend.withColumnRenamed("merchant_name","key")
final_score = final_score.join(df_trend, df_trend.key ==  revenue_score.merchant_name,"inner").drop("key")
final_score

merchant_name,category,merchant_rev_score,category_rev_score,fraud_score,trend_score
Tincidunt Aliquam...,art and gifts,0.0,0.0,1.0,-0.29196876218275214
Amet Ultricies Se...,art and gifts,0.0,0.0,0.9966666666666667,-0.2870785906746487
Eleifend Nunc LLP,art and gifts,5.242463958060288E-4,0.003780718336483932,1.0,-0.2850052340899205
Vel Nisl Incorpor...,art and gifts,7.863695937090433E-4,0.005671077504725898,0.9970588235294118,-0.27740917670801746
Penatibus Et Magn...,art and gifts,0.001048492791612...,0.007561436672967864,1.0,-0.27400122250493186
Eget Volutpat Orn...,art and gifts,0.001310615989515072,0.00945179584120983,1.0,-0.2951705163690119
Nulla Semper LLP,art and gifts,0.001572739187418...,0.011342155009451797,1.0,-0.28928102865155325
In Lorem LLP,art and gifts,0.001834862385321101,0.013232514177693762,0.9962406015037594,-0.29661931434770994
Ut Erat PC,art and gifts,0.002096985583224...,0.015122873345935728,0.9978260869565218,-0.2935104146561206
Volutpat Ornare LLP,art and gifts,0.00235910878112713,0.017013232514177693,1.0,-0.2851980847309472


In [271]:
df_trend.sort(F.col("trend_score"))

key,trend_score
Rutrum Magna Cras...,-1.0
Sed Nec Inc.,-0.91774036166771
Pharetra Corp.,-0.8334200386655441
Fames Ac Turpis LLC,-0.6752184490530835
Amet Risus Inc.,-0.6596795683980416
Dignissim Lacus PC,-0.6399996588754957
In Lobortis Tellu...,-0.6257353623479553
Id Enim Inc.,-0.625085997402387
Ut Ipsum LLC,-0.6037409750782243
At Pede Inc.,-0.5833355926027


## 4. Stability score (0~1)

In [272]:
df_variance = df_old.groupBy(["merchant_name", "order_year", "order_month"]).agg(
    sum("dollar_value").alias("dollar_value")
    )

In [273]:
fill = {'2021': range(3,13),
        '2022': range(1,11)}
merchs = df_variance.select("merchant_name").distinct()
cols = ["order_year", "order_month", "dollar_value"]
vals = [(int(year), month, 0) for year in fill for month in fill[year]]
months = spark.createDataFrame(vals, cols)
months = merchs.join(months).select(df_variance.columns)
variance_score = df_variance.union(months)

In [274]:
variance_score

merchant_name,order_year,order_month,dollar_value
Lacinia At Institute,2022,8,11473.893084260102
Phasellus At Limited,2021,6,365871.4302617312
Eget Laoreet Posu...,2022,8,179542.76522072856
Eu Tempor Ltd,2021,4,5001.601112401771
Tempor LLC,2021,12,10105.609794197258
Semper Tellus PC,2022,6,36826.62851667091
Ac Institute,2021,6,34412.903467209915
Mattis Velit Just...,2021,3,98927.17618829764
Lacinia Mattis LLC,2022,5,55866.78480219141
Aliquam Gravida M...,2021,12,50557.11317859411


In [275]:
variance_score = variance_score.groupBy(["merchant_name", "order_year", "order_month"]).agg(
    sum("dollar_value").alias("dollar_value")
    )

In [276]:
variance_score = variance_score.groupBy(["merchant_name"]).agg(
    variance("dollar_value").alias("variance")
    )

In [277]:
variance_score

merchant_name,variance
Malesuada Vel Ltd,4456926.807088791
Mollis Integer Co...,4.582009408526634E7
Volutpat Nulla In...,1.9624456909507588E8
Donec Luctus Indu...,3.323628064942223E8
Vulputate Velit E...,725151.9694819615
Erat Semper Ltd,1.2301800075294748E8
Hendrerit Consect...,1.7210253143555377E7
Vel Turpis Company,4287985.3046595715
Curabitur Vel LLC,333323.39801631885
Malesuada PC,1973086.758359104


In [278]:
variance_score = variance_score.fillna(value = 0)

In [279]:
# VectorAssembler Transformation - Converting column to vector type
assembler_vec = VectorAssembler(inputCols=["variance"],outputCol="variance_Vect")
variance_score = assembler_vec.transform(variance_score)

scaler = MinMaxScaler(inputCol="variance_Vect", outputCol="variance_score")


# Standardise the trend score (since it contain negative value) as score
from pyspark.ml.functions import vector_to_array
variance_score = scaler.fit(variance_score).transform(variance_score).drop("variance","variance_Vect").withColumn("variance_score", vector_to_array("variance_score")[0])


In [280]:
variance_score = variance_score.withColumnRenamed("merchant_name","key")
final_score = final_score.join(variance_score, variance_score.key ==  revenue_score.merchant_name,"inner").drop("key")
final_score

merchant_name,category,merchant_rev_score,category_rev_score,fraud_score,trend_score,variance_score
Malesuada Vel Ltd,books and music,0.6482306684141547,0.6735436893203883,0.9968982630272953,-0.2659155054587914,3.298045669878045E-4
Mollis Integer Co...,books and music,0.8681520314547837,0.8847087378640777,0.9988385598141696,-0.22634669382732775,0.003390957761643...
Volutpat Nulla In...,home and technology,0.9698558322411533,0.9711538461538461,0.9984037034080933,-0.24845221461810907,0.014523382876334779
Donec Luctus Indu...,fashion and acces...,0.6422018348623854,0.5954415954415955,0.9975812953507122,-0.2821378659840347,0.02459704997399887
Vulputate Velit E...,fashion and acces...,0.2566186107470511,0.2378917378917379,1.0,-0.28624214187783215,5.362807428715451...
Erat Semper Ltd,outdoors,0.9155963302752294,0.9036585365853659,0.9973759541984732,-0.2705380856818037,0.009104123221804636
Hendrerit Consect...,books and music,0.8125819134993447,0.8337378640776699,0.9981889105600446,-0.3230723102625034,0.001273636682038701
Vel Turpis Company,home and technology,0.46579292267365663,0.5021367521367521,0.9953416149068323,-0.3488128370442415,3.173017570266328...
Curabitur Vel LLC,books and music,0.3255570117955439,0.32402912621359226,1.0,-0.27967123028464425,2.463011880582002E-5
Malesuada PC,outdoors,0.4301441677588467,0.38414634146341464,0.997093023255814,-0.31026056064007324,1.459836605990655...


In [281]:
final_score.count()

3816

In [282]:
final_score.write.parquet("../data/curated/score_table")

22/10/10 02:01:23 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 210350 ms exceeds timeout 120000 ms
22/10/10 02:01:28 WARN SparkContext: Killing executors is not supported by current scheduler.
